In [1]:
import pandas as pd
import cv2
import os
import tqdm

In [5]:
import glob

file_list = glob.glob('/home/user/Helipad_Detection/workspace/annotations/stanford_raw/*.csv')

for file in file_list:
    t = file.split('/')[-1].split('_')[0]
    
    image_path = f'/home/user/Helipad_Detection/workspace/images/stanford/{t}'
    data = pd.read_csv(file)
    
    before_cols = ['filename','xmin','ymin','xmax','ymax', 'class']
    data.columns = before_cols
    data['filename'] = data['filename'].apply(lambda x : x.split('/')[-1])
    
    size_dic = {}
    for fnm in tqdm.tqdm(data['filename'].unique()):
        height, width, _= cv2.imread(os.path.join(image_path, fnm)).shape
        size_dic[fnm] = (height, width)
        
    data['height'] = list(map(lambda x: size_dic[x][0], data['filename']))
    data['width'] = list(map(lambda x: size_dic[x][1], data['filename']))
    
    after_cols = ['filename','width','height','class','xmin','ymin','xmax','ymax']
    data[after_cols].to_csv('/home/user/Helipad_Detection/workspace/annotations/' + file.split('/')[-1], index=False)

100%|█████████████████████████████████████████| 663/663 [00:13<00:00, 48.11it/s]


In [8]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

In [6]:
def class_text_to_int(row_label):
    return label_map_dict[row_label]

In [7]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [9]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [29]:
file_list = glob.glob('/home/user/Helipad_Detection/workspace/annotations/*_stanford.csv')
path = '/home/user/Helipad_Detection/workspace/images/stanford'

In [30]:
label_map = label_map_util.load_labelmap('/home/user/Helipad_Detection/workspace/annotations/label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [32]:
for t in ['train', 'val', 'test']:
    file = f'/home/user/Helipad_Detection/workspace/annotations/{t}_stanford.csv'
    path = f'/home/user/Helipad_Detection/workspace/images/stanford/{t}'
    output_path = f'/home/user/Helipad_Detection/workspace/annotations/{t}_stanford.record'
    writer = tf.python_io.TFRecordWriter(output_path)
    
    examples = pd.read_csv(file)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(output_path))

Successfully created the TFRecord file: /home/user/Helipad_Detection/workspace/annotations/train_stanford.record
Successfully created the TFRecord file: /home/user/Helipad_Detection/workspace/annotations/val_stanford.record
Successfully created the TFRecord file: /home/user/Helipad_Detection/workspace/annotations/test_stanford.record
